In [5]:
import math
import pandas as pd
import numpy as np
from pprint import pprint

from scipy import constants
from astropy import constants as astro_const
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

import pbhs as pbhs
import black_holes as bh

pd.set_option('display.width', 500)

# solar_mass_grams = 1000 * astro_const.M_sun.value

# PBHs

## Quantum evaporation and absorption of energy

Black holes emit thermal radiation at a temperature inversely proportional to their mass. Numerically, this effect is given by

\begin{equation}
T_{bh}=\frac{\hbar{c}^{3}}{8\pi{k_{B}G}M}\simeq \frac{6.2\times{10}^{-8}K}{(M/M_{\odot})}
\end{equation}




In [6]:
solar_mass_units = np.arange(1, 5)

masses_kg = solar_mass_units * astro_const.M_sun.value

# Calculate the Hawking temperature for each mass
hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, astro_const.M_sun.value) for m in masses_kg]

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_solar) from import': solar_mass_units,
    'Mass (kg)': masses_kg,
    'Hawking Temperature (K)': hawking_temperatures
})

# Display the table
print(df)

   Mass (M_solar) from import     Mass (kg)  Hawking Temperature (K)
0                           1  1.988410e+30             6.200000e-08
1                           2  3.976820e+30             3.100000e-08
2                           3  5.965230e+30             2.066667e-08
3                           4  7.953639e+30             1.550000e-08


In [7]:
temperature_critical_mass = pbhs.critical_mass_for_bg_temperature(2.7) / pbhs.solar_mass_grams
dmdt, first, second, _ = pbhs.dm_dt(5 * pbhs.solar_mass_grams, 2.7)

print(f"mass_c({temperature_critical_mass} K): {temperature_critical_mass:.2e}")
print(f"dmdt: {dmdt:.2e}")
print(f" - first: {first:.2e}")
print(f" - second: {second:.2e}")


mass_c(2.2910545872296856e-08 K): 2.29e-08
dmdt: 9.19e-11
 - first: -4.01e-44
 - second: 9.19e-11


In [14]:
background_temperature = 2.7
# solar_mass_units = np.arange(1, 100)
solar_mass_units = np.logspace(start=-20, stop=1, num=50, base=10)


temperature_critical_mass = pbhs.critical_mass_for_bg_temperature(background_temperature)
print(96*"-")
print(f"Planck mass (g): {pbhs.planck_mass_g:.2e} grams")
print(f"critical_mass({background_temperature} K): {temperature_critical_mass/pbhs.solar_mass_grams:.2e} \M_sun ({temperature_critical_mass:.2e} grams)")
print(96*"-")

masses_g = solar_mass_units * pbhs.solar_mass_grams
# print("\nMasses(g)")
# pprint(masses_g)

hawking_temperatures = [pbhs.hawking_temp_from_solar_mass(m, pbhs.solar_mass_grams) for m in masses_g]
# print("\nHawking Temperatures(K)")
# pprint(hawking_temperatures)

terms_tuple = [pbhs.dm_dt(m, background_temperature) for m in masses_g]
# print("\nterms_tuple")
# pprint(terms_tuple)

dm_dt_column, first_term_col, second_term_col, error_diff = zip(*terms_tuple)
# print("\ndm_dt_column")
# pprint(dm_dt_column[0])

# Create a DataFrame to display the results in a table
df = pd.DataFrame({
    'Mass (M_sun)': solar_mass_units,
    'Mass (g)': masses_g,
    'Hawking temp. (K)': hawking_temperatures,
    # 'dM/dt': dm_dt_column,
    'First term (Hawking Evaporation)': first_term_col,
    'Second term (absorption)': second_term_col,
    'Close?': error_diff
})

# Display the table
print(df.to_markdown(floatfmt='.2e', index=False))

------------------------------------------------------------------------------------------------
Planck mass (g): 2.18e-05 grams
critical_mass(2.7 K): 2.29e-08 \M_sun (4.56e+25 grams)
------------------------------------------------------------------------------------------------
|   Mass (M_sun) |   Mass (g) |   Hawking temp. (K) |   First term (Hawking Evaporation) |   Second term (absorption) | Close?   |
|---------------:|-----------:|--------------------:|-----------------------------------:|---------------------------:|:---------|
|       1.00e-20 |   1.99e+13 |            6.20e+12 |                          -1.00e-02 |                   3.68e-52 | False    |
|       2.68e-20 |   5.33e+13 |            2.31e+12 |                          -1.39e-03 |                   2.65e-51 | False    |
|       7.20e-20 |   1.43e+14 |            8.61e+11 |                          -1.93e-04 |                   1.90e-50 | False    |
|       1.93e-19 |   3.84e+14 |            3.21e+11 |           

<>:9: SyntaxWarning: invalid escape sequence '\M'
<>:9: SyntaxWarning: invalid escape sequence '\M'
/var/folders/n5/91crw72j2wl9689p75btgrf40000gn/T/ipykernel_47398/2896665515.py:9: SyntaxWarning: invalid escape sequence '\M'
  print(f"critical_mass({background_temperature} K): {temperature_critical_mass/pbhs.solar_mass_grams:.2e} \M_sun ({temperature_critical_mass:.2e} grams)")


# PBHs as Dark Matter candidates

| Mass Range (Approximate)  |  Source of Constraint/Window |  Key Observations/Phenomena |
|---|---|---|
| $< 10^{15}$ g ($\sim 10^{-18} M_{\odot}$) | Excluded| Hawking Radiation: Black holes lighter than this mass would have completely evaporated by the present age of the Universe ($\sim 13.8$ billion years).|
| $10^{15} \text{ g} - 10^{20} \text{ g}$ | Severely Constrained | Gamma-ray/CMB distortions, Big Bang Nucleosynthesis (BBN): Evaporation of PBHs in this range before or during BBN or reionization would inject high-energy particles, altering the observed light element abundances and the Cosmic Microwave Background (CMB) spectrum. |
| $\sim 10^{20} \text{ g} - 10^{24} \text{ g}$ ($\sim 10^{-13} - 10^{-9} M_{\odot}$, Asteroid Mass) | Open Window | This range is currently one of the most unconstrained, often referred to as the asteroid-mass window. Constraints from microlensing and other effects are weaker here. |
| $\sim 10^{-9} M_{\odot} - 10^{-2} M_{\odot}$ | Severely Constrained | Microlensing Surveys (e.g., MACHO, EROS): Observing the temporary brightening of distant stars due to the gravitational lensing effect of massive objects (PBHs) passing in front of them provides strong limits on PBHs in this range (up to a few $M_{\odot}$). |
| $\sim 10 M_{\odot} - 100 M_{\odot}$ (Stellar Mass)| Controversial/Reopened Window | Gravitational Waves (LIGO/Virgo/KAGRA): PBH mergers in this range are consistent with the observed merger rates, re-opening this window, but constraints from microlensing and dynamical friction are still debated and may require a non-monochromatic (extended) mass distribution. |
| $\sim 10^3 M_{\odot} - 10^5 M_{\odot}$ | Constrained | Dynamical constraints, effects on stellar clusters: Massive PBHs would affect the dynamics of globular clusters and ultra-faint dwarf galaxies. |
|$\gg 10^5 M_{\odot}$ | Severely Constrained | CMB, Large-Scale Structure (LSS): Very massive PBHs would significantly perturb the LSS formation and induce too many anisotropies in the CMB. |



# The Expected PBH Mass Distribution

If PBHs account for all DM, their distribution in mass, $\psi(M)$, is not expected to be monochromatic (all PBHs having the same mass) due to the nature of their formation from primordial density fluctuations.

The most commonly considered theoretical mass distribution for PBHs formed from a smooth, narrow peak in the primordial power spectrum is the log-normal distribution:

$$\psi(M) \propto \frac{1}{M \sigma} \exp \left( -\frac{\ln^2(M/M_c)}{2\sigma^2} \right)$$

* $M$: PBH mass
* $M_c$: Peak (characteristic) mass of the distribution
* $\sigma$: Width of the distribution

This extended distribution means that if the peak mass $M_c$ falls in an "open window" (like the asteroid-mass range), the tails of the distribution must still satisfy the constraints in other mass ranges.

The recent consensus, especially motivated by Gravitational Wave observations and the need to evade various constraints, suggests that a PBH DM scenario requires a broad mass distribution (large $\sigma$) that spans several orders of magnitude, with the majority of the mass concentrated in the open windows, such as the asteroid-mass window ($\sim 10^{20} - 10^{24} \text{ g}$), and possibly the stellar-mass window ($\sim 10 - 100 M_{\odot}$).

In summary, the "calculation" points towards an allowed mass distribution that is:

* Entirely above $\sim 10^{15}$ g to avoid complete evaporation.
* Highly suppressed in the ranges strongly constrained by microlensing ($10^{-2} M_{\odot} - 10 M_{\odot}$).
* Dominated by mass in the open windows, particularly the asteroid-mass range, to constitute the majority of dark matter.

# Asteroid-mass window as source of all dark matter

If Primordial Black Holes (PBHs) in the asteroid-mass window are the source of all dark matter, the required number density in a volume of 1 cubic light-year (ly³) is surprisingly small.

The calculation involves two main steps:
1. Determine the local dark matter mass density ($\rho_{\text{DM}}$)
2. Divide the mass density by the PBH mass (1$M_{\text{PBH}}$) to find the number density (2$n_{\text{PBH}}$)

## Local Dark Matter Mass Density ($\rho_{\text{DM}}$)

The mass density of dark matter in the solar neighborhood is widely estimated to be:

$$\rho_{\text{DM}} \approx 0.3 - 0.4 \text{ GeV/cm}^3$$

To translate this into solar masses per cubic light-year, we use the following conversions:

* $1 \text{ GeV/cm}^3 \approx 5.35 \times 10^{-4} M_{\odot}/\text{pc}^3$
* $1 \text{ pc} \approx 3.26 \text{ ly}$
* $1 \text{ pc}^3 \approx 3.26^3 \text{ ly}^3 \approx 34.6 \text{ ly}^3$

Using a fiducial value of $\rho_{\text{DM}} \approx 0.4 \text{ GeV/cm}^3$:

$$\rho_{\text{DM}} \approx \frac{0.4 \text{ GeV/cm}^3 \times 5.35 \times 10^{-4} M_{\odot}/\text{pc}^3}{1 \text{ GeV/cm}^3} \approx 2.14 \times 10^{-4} M_{\odot}/\text{pc}^3$$

Now, convert to $M_{\odot}/\text{ly}^3$:

$$\rho_{\text{DM}} \approx \frac{2.14 \times 10^{-4} M_{\odot}}{\text{pc}^3} \times \frac{1 \text{ pc}^3}{34.6 \text{ ly}^3}$$

$$\mathbf{\rho_{\text{DM}} \approx 6.2 \times 10^{-6} \text{ Solar Masses per Cubic Light-Year}}$$

## PBH Number Density ($n_{\text{PBH}}$)

The asteroid-mass window is typically cited as $10^{17} \text{ g}$ to $10^{23} \text{ g}$ .

We'll calculate the number of PBHs per cubic light-year ($N_{\text{PBH}}$) for the extremes of this range and a central value.

The formula for the number density is:

$$N_{\text{PBH}} = \frac{\rho_{\text{DM}} \text{ (in kg/ly}^3)}{M_{\text{PBH}} \text{ (in kg)}}$$

First, convert the dark matter mass to kilograms:

* $1 M_{\odot} \approx 2 \times 10^{30} \text{ kg}$

* $\rho_{\text{DM}} \approx 6.2 \times 10^{-6} M_{\odot}/\text{ly}^3 \approx (6.2 \times 10^{-6}) \times (2 \times 10^{30}) \text{ kg}/\text{ly}^3 \approx \mathbf{1.24 \times 10^{25} \text{ kg/ly}^3}$

### Resulting Distribution

| PBH Mass (MPBH​) | PBH Mass in Kilograms | Number of PBHs per ly³ (NPBH​) |
|---|---|---|
|Lower End ($10^{17} \text{ g}$)     | $10^{14} \text{ kg}$ | $\frac{1.24 \times 10^{25} \text{ kg/ly}^3}{10^{14} \text{ kg}} \approx \mathbf{1.24 \times 10^{11}}$ (124 Billion)
|Central Value ($10^{20} \text{ g}$) | $10^{17} \text{ kg}$ | $\frac{1.24 \times 10^{25} \text{ kg/ly}^3}{10^{17} \text{ kg}} \approx \mathbf{1.24 \times 10^8}$ (124 Million)
|Upper End ($10^{23} \text{ g}$)     | $10^{20} \text{ kg}$ | $\frac{1.24 \times 10^{25} \text{ kg/ly}^3}{10^{20} \text{ kg}} \approx \mathbf{124,000}$


### Conclusion

For PBHs in the asteroid-mass window to account for all of the dark matter, their number density in a single cubic light-year of space must be between approximately 124,000 (for the heaviest PBHs) and 124 billion (for the lightest PBHs).This high number density suggests that the PBHs would be extremely numerous but individually very light compared to typical astrophysical objects. They would be spread out but constantly zipping through the Solar System and surrounding space, which is why future missions are looking for subtle gravitational perturbations from these hypothetical objects.



# fitar os pbhs no halo da galaxia
# analogia com 21cm, emissao hawking
# Onda gravitacional
# Efeitos estocasticos?
# Distancia media
# ja foi feito por alguem?

# Accepted Milky Way Dark Matter Halo Parameters

The distribution and size of the Milky Way's dark matter (DM) halo are not uniquely defined, as different models (cored vs. cuspy) and observational tracers yield variations. However, a Navarro-Frenk-White (NFW) profile is the most widely cited in the $\Lambda$CDM framework, while the Burkert profile is often favored by fits to observed rotation curves in low-surface-brightness and some high-surface-brightness galaxies, including the Milky Way.

## Canonical Halo Model (for calculation)

For our insights, we will adopt the standard NFW parameters and the most common local DM density estimate, as these are frequently used in PBH constraint literature.

* Halo Profile (NFW): The mass density $\rho_{\text{DM}}(r)$ at a galactocentric radius $r$ is given by:

$$\rho_{\text{NFW}}(r) = \frac{\rho_s}{\frac{r}{r_s} \left(1 + \frac{r}{r_s}\right)^2}$$

where $\rho_s$ is the characteristic density and $r_s$ is the scale radius.

* Local Dark Matter Density ($\rho_\odot$): The accepted value at the Sun's position ($R_\odot \approx 8.122$ kpc) is approximately:

$$\rho_{\text{DM}}(R_\odot) \approx 0.3 - 0.5 \text{ GeV/cm}^3 \approx 4.0 - 6.7 \times 10^{-22} \text{ g/cm}^3$$

We will use a canonical value of:

$$\mathbf{\rho_\odot = 0.4 \text{ GeV/cm}^3} \approx 6.75 \times 10^{-25} \text{ g/cm}^3$$

Note: The NFW scale radius ($r_s$) for the Milky Way is typically $\sim 15 - 20$ kpc, and the virial radius ($R_{\text{vir}}$, where the mean density is $200\times\rho_{\text{crit}}$) is $\sim 200$ kpc, enclosing a total mass $M_{\text{vir}} \sim 10^{12} M_\odot$.